In [ ]:
from IPython.display import clear_output

import subprocess
import os
import sys
import glob
from multiprocessing import Process, Queue
import pandas as pd
import optuna
import time
import numpy as np
sys.path.append('/opt/conda/GSASII/')
import re
import datetime
import GSASIIscriptable as G2sc

DATABASE_DIR = "/bbo_rietveld/database"

In [2]:
def bbor_run(target_file_path):
    target_file_name = os.path.basename(target_file_path)
    class ProjectTiO2:
        def __init__(self, trial_number):
            self.gpx = G2sc.G2Project(newgpx=os.path.join(WORK_DIR, "trial{}_{}.gpx".format(trial_number,newname)))

            G2sc.SetPrintLevel('none')

            self.hist1 = self.gpx.add_powder_histogram(
                target_file_path, os.path.join(DATABASE_DIR, 'Lab_CuKa.instprm'), fmthint='xye')

            self.phase0 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'Anatase.cif'),
                                            phasename='anatase',
                                            histograms=[self.hist1])
            self.phase1 = self.gpx.add_phase(os.path.join(DATABASE_DIR, 'Rutile.cif'),
                                            phasename='rutile',
                                            histograms=[self.hist1])

            self.hist1.data['Sample Parameters']['Scale'][1] = False
            self.gpx.data['Controls']['data']['max cyc'] = 10
            self.phase0.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True
            self.phase1.data['Histograms']['PWDR '+target_file_name]['Scale'][1] = True

            for val in self.phase0.data['Atoms']:
                val[9] = 'I'
            for val in self.phase1.data['Atoms']:
                val[9] = 'I'

        def refine_and_calc_Rwp(self, param_dict):
            self.gpx.do_refinements(refinements=[param_dict])
            for hist in self.gpx.histograms():
                _, Rwp = hist.name, hist.get_wR()
            return Rwp

    def objective(trial):
        sample_parameters0_refine =[]
        for p in ['phase0']:
            if not trial.suggest_categorical('Phase_parameters refine %s' % (p), [True, False]):
                sample_parameters0_refine.append(p)
        not_ref_phases = sample_parameters0_refine
        refdict0 = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}

        background_type = trial.suggest_categorical(
            'Background type', ['chebyschev',
                                'chebyschev-1',
                                'cosine',
                                'Q^2 power series',
                                'Q^-2 power series'
                                ])
        no_coeffs = trial.suggest_int('Number of coefficients', 10, 16)
        background_refine = trial.suggest_categorical('Background refine', [True, False])
        refdict0bg_h = {
            'set': {
                'Background': {
                    'type': background_type,
                    'no. coeffs': no_coeffs,
                    'refine': background_refine
                }
            }
        }

        instrument_parameters1_refine = []
        for p in ['Zero']:
            if trial.suggest_categorical('Instrument_parameters refine %s' % (p), [True, False]):
                instrument_parameters1_refine.append(p)
        refdict1_h = {'set': {'Cell': True, 'Instrument Parameters': instrument_parameters1_refine}}

        sample_parameters1_refine =[]
        for p in ['Shift', 'SurfRoughA','SurfRoughB']:
            if trial.suggest_categorical('Sample_parameters refine %s' % (p), [True, False]):
                sample_parameters1_refine.append(p)
        refdict1_h2 = {"set": {'Sample Parameters':sample_parameters1_refine }}

        instrument_parameters2_refine = []
        for p in ['U', 'V', 'W', 'X', 'Y', 'SH/L']:
            if trial.suggest_categorical('Peakshape_parameters refine %s' % (p), [True, False]):
                instrument_parameters2_refine.append(p)
        refdict2_h = {'set': {'Instrument Parameters': instrument_parameters2_refine}}

        refdict3_h_2 = {'set': {'Atoms': {'all': 'XU'}}}

        mustrain_refine = trial.suggest_categorical('Mustrain refine', [True, False])
        refdict_new  ={
            'set':{
                'Mustrain':{
                    'type':'uniaxial',
                    'direction': [0, 0, 1],
                    'refine': mustrain_refine
                }
            }
        }

        refdict_fin_h = {'set': {'Limits': [twot_srart, twot_end]}, 'refine': True}

        refine_params_list = [
            not_ref_phases,
            refdict0,
            refdict0bg_h,
            refdict1_h,
            refdict1_h2,
            refdict2_h,
            refdict3_h_2,
            refdict_fin_h,
            refdict_new
            ]

        def evaluate(trial_number, refine_params_list, q):
            ERROR_PENALTY = 1e9
            try:
                project = ProjectTiO2(trial_number)
                for leng in range(len(project.gpx.phases())):
                    project.gpx.phases()[leng].data['pId'] = leng
                for params in refine_params_list[1:]:
                    Rwp = project.refine_and_calc_Rwp(params)
                for i in range(len(project.gpx.phases())):
                    phase_TiO2 = project.gpx.phases()[i]
                    u_iso_list = [atom.uiso for atom in phase_TiO2.atoms()]
                scale_list = [i['Histograms']['PWDR '+target_file_name]['Scale'][0] for i in project.gpx.phases()]
                if min(scale_list) < 0.001:
                    Rwp = ERROR_PENALTY
                q.put(Rwp)

            except Exception as e:
                q.put(ERROR_PENALTY)

        q = Queue()
        p = Process(target=evaluate, args=(trial.number, refine_params_list, q))
        p.start()
        Rwp = q.get()
        p.join()

        return Rwp

    start_time = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
    newname = "{}".format(target_file_name.strip(".csv"))
    study = optuna.create_study(study_name=newname
                            ,sampler=optuna.samplers.TPESampler(n_startup_trials=random_trials, seed=RANDOM_SEED))
    start = time.time()
    study.optimize(objective, n_trials=trials, n_jobs=jobs)
    end = time.time()-start
    df = study.trials_dataframe()
    df.columns = [' '.join(col).replace('params', '').strip().replace(' ','') for col in df.columns.values]
    df.rename(columns={'value':'Rwp', 'number':'trial'}, inplace=True)
    df.to_pickle(os.path.join(WORK_DIR,newname+".pkl"))

    best_trial_number = study.best_trial.number
    best_rwp = int(study.best_trial.value)

    new_dir_name = newname + start_time.strftime('_%Y%m%d_%H%M%S') + "_trial{}".format(best_trial_number) + "_Rwp{}".format(best_rwp)
    newdir = os.path.join(WORK_DIR, new_dir_name)

    result_path = os.path.join(RESULT_DIR, new_dir_name)
    subprocess.run("mkdir {}".format(result_path), shell=True)

    df.to_csv(result_path + "/result.csv", mode='w')
    subprocess.run("cp {} {}".format(target_file_path, result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.lst".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {} {}".format(os.path.join(WORK_DIR,"trial{}_{}.gpx".format(study.best_trial.number,newname)), result_path), shell=True)
    subprocess.run("cp {}/*.pkl {}".format(WORK_DIR,result_path), shell=True)

    subprocess.run("mkdir {}".format(newdir), shell=True)
    subprocess.run("mv {}/*.gpx {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.lst {}".format(WORK_DIR,newdir), shell=True)
    subprocess.run("mv {}/*.pkl {}".format(WORK_DIR,newdir), shell=True)

    data_path = os.path.join(result_path,"trial{}_{}.gpx".format(study.best_trial.number,newname))
    gpx = G2sc.G2Project(data_path)
    for i,h in enumerate(gpx.histograms()):
        if i > 0:
            raise Exception ('Multiple histograms is not applicable')
        hfil = os.path.splitext(data_path)[0]
        if os.path.exists(hfil+'.csv'):
            print ('{}.csv is already existed. No need to run this'.format(hfil))
            return
        print(h.name, "analyzed"+hfil+'.csv')
        h.Export(hfil,'.csv','histogram CSV')

    clear_output(True)

    print("Finished!!")
    print("Best trial:" + str(study.best_trial.number))
    print("Best Rwp:" + str(study.best_trial.value))
    print("Time: {:.2f} sec".format(end), " ({})".format(datetime.timedelta(seconds=end)))

In [5]:
### SET YOUR PARAMETER ###
STUDY_NAME = 'TiO2'
target_dir = "/bbo_rietveld/XRD_data"

twot_srart = 20
twot_end = 120

RANDOM_SEED =  1024
random_trials = 20
trials = 200
jobs = 20

now = datetime.datetime.now()
today = now.strftime('%Y%m%d')
WORK_DIR = '/bbo_rietveld/work/' + STUDY_NAME + '/' + today
RESULT_DIR = '/bbo_rietveld/result/' + STUDY_NAME + '/' + today

In [ ]:
file_list = glob.glob(target_dir + "/*.csv")
print("file =",len(file_list))
print(glob.glob(target_dir + "/*.csv"))

In [7]:
# make work directory
! mkdir -p $WORK_DIR
! mkdir -p $RESULT_DIR

In [ ]:
analyzed = []
while True:
    file_list = glob.glob(target_dir + "/*.csv")
    unanalyzed = [f for f in file_list if f not in analyzed]
    if len(unanalyzed) > 0:
        unanalyzed.sort()
        target_file = unanalyzed[0]
        print("target: ",target_file)
        bbor_run(target_file)
        analyzed.append(target_file)
        print("analyzed: ",analyzed)
    else:
        time.sleep(60)